<a href="https://colab.research.google.com/github/ArveloJuan/gitbrachmerge/blob/main/hw6.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import statement with all relevant packages.**

In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot colorcet datashader bebi103 watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    data_path = "https://s3.amazonaws.com/bebi103.caltech.edu/data/"
else:
    data_path = "../data/"
# ------------------------------
import numpy as np
import pandas as pd

import iqplot

import math
import bebi103 as be
import warnings

import tqdm

import scipy.optimize
import scipy.stats as st

**Creates a data frame with the required data but also goes ahead and takes the average of the parents for the fortis set which does not have the proper column already and relabels the offspring column to match across both dataframes.**

In [30]:
fortis_df = pd.read_csv(os.path.join(data_path, "fortis_beak_depth_heredity.csv"), skiprows=3)
scandens_df = pd.read_csv(os.path.join(data_path, "scandens_beak_depth_heredity.csv"), skiprows=3)

#Data wrangling so that both dfs have a "mid_parent" and "mid_offspring" column
fortis_df["mid_parent"] = (fortis_df["Male BD"] + fortis_df["Female BD"])/2
fortis_df.rename(columns = {"Mid-offspr": "mid_offspring"}, inplace = True)


**These functions set up the definitions needed to run the mle function later on by properly unpacking parameters into a log likelihood function for the bivariate normal distribution. This code also goes ahead and ignores inapproriate sigma matrixes that would not work. The second definition uses optimize to find the MLEs. Here we assume that the input of variables will be mus followed by sigmas in reading order without inputting the repeat. The reason we have put this condition of the sigma matrix having to be positive is because a negative variance is meaningless in probability so we should avoid them.**

In [31]:
def log_likelihood(params, data):
    """
    converts array of parameters into the inputs for multivariate normal distribution
    """
    
    mu = params[0:2]
    sigma = [[params[2], params[3]],
             [params[3], params[4]]]
             
    if(np.linalg.det(sigma) <= 0):
        return -np.inf
    return np.sum(scipy.stats.multivariate_normal.logpdf(data, mu, sigma))

In [32]:
def parametric_mle(data):
    '''
    Computes a parametric mle for the "mid_parent" and "mid_offspring" values of a dataframe
    '''  
    
    #data = np.asarray(data)
    #print(data[0])
    #print(data)
    d = data[:, 0]
    ell = data[:, 1]
    #d = df['mid_parent'].values
    #ell = df['mid_offspring'].values
    #arr = df[["mid_parent", "mid_offspring"]].values,
    res = scipy.optimize.minimize(
        fun = lambda params, data: -log_likelihood(params, data), 
        x0 = np.array([1,1,1,0,1]), #the matrix must be positive definite
        args = (data),
        method = 'Powell'
    )
    return res.x

**Using the definition provided in the notes this function calculates the heredity value. This value is defined as $h^2 = \frac{σ_{op}}{σ_p^2}$. Here we assume the organization of the parameter list again.**

In [33]:
def heredity(params):
    '''
    Converts 1d array of parameters into a measure of heretability
    ''' 
    
    return (params[3]/params[2])

**Here we use the definitions above and actually supply them a dataset to operate on.**

In [34]:
data = fortis_df[["mid_parent", "mid_offspring"]].values

fortis_mle = parametric_mle(data)
#scandens_mle = parametric_mle(scandens_df)w

In [35]:
fortis_mle

array([9.48427361, 9.3046489 , 0.4763311 , 0.34438081, 0.46927458])

**Above in this array is our parametric likelihood estimates for the parameters of the matrix. The first two values are the means and the next four are the variances needed to construct the matrix in reading order omitting the repeated variance.**

In [36]:
heredity(fortis_mle)

0.7229861914395019

**This is our calculated value for the heredity of beak depth across generations this is represented as the ratio of covariances between the parents and children versus the parents alone as described above.**

**Code block here defines the generating function and bootstraping function together for simplicity to construct confidence intervals.**

In [37]:
def genfun(*params, size):
    """
    This function generates the bivariate normal distribution based on the log
    of parameters.
    """
    mu = params[0:2]
    sigma = [[params[2], params[3]],
             [params[3], params[4]]]
    return  scipy.stats.multivariate_normal.rvs(mu, sigma, size=size)

def draw_parametric_bs_reps_mle(
    mle_fun, gen_fun, data, args=(), size=1, progress_bar=False
):
    """Draw parametric bootstrap replicates of maximum likelihood estimator.

    Parameters
    ----------
    mle_fun : function
        Function with call signature mle_fun(data, *args) that computes
        a MLE for the parameters
    gen_fun : function
        Function to randomly draw a new data set out of the model
        distribution parametrized by the MLE. Must have call
        signature `gen_fun(*params, size)`.
    data : one-dimemsional Numpy array
        Array of measurements
    args : tuple, default ()
        Arguments to be passed to `mle_fun()`.
    size : int, default 1
        Number of bootstrap replicates to draw.
    progress_bar : bool, default False
        Whether or not to display progress bar.

    Returns
    -------
    output : numpy array
        Bootstrap replicates of MLEs.
    """
    params = mle_fun(data)

    if progress_bar:
        iterator = tqdm.tqdm(range(size))
    else:
        iterator = range(size)

    return np.array(
        [mle_fun(gen_fun(*params, size=len(data))) for _ in iterator]
    )

**This code implements the function above and constructs a confidence interval from the bootstrap samples. The first confidence is for heredity as stated. The second is for the two means and then the variances in reading form ignoring the repeat. The first array is the low bound for these terms in this order and the second the high bound.** 

In [45]:
bs_reps_parametric = draw_parametric_bs_reps_mle(
    parametric_mle,
    genfun,
    data=fortis_df[["mid_parent", "mid_offspring"]].values,
    args=(),
    size=100,
    progress_bar=False,
)

CI = np.percentile(bs_reps_parametric, [2.5, 97.5], axis=0)
low_bo = np.percentile(bs_reps_parametric, [2.5], axis=0)
high_bo = np.percentile(bs_reps_parametric, [97.5], axis=0)
low_hered = low_bo[0][3]/ low_bo[0][2]
high_hered = high_bo[0][3]/low_bo[0][2]

print('our confidence interval for the heredity h squared is: ' + str(low_hered)
      + " to " +str(high_hered)+ '\n')
print(CI)

our confidence interval for the heredity h squared is: 0.7125032820678228 to 0.9728471431298831

[[9.41776686 9.23745286 0.40822864 0.29086425 0.40320915]
 [9.55713477 9.36018782 0.54326249 0.39714407 0.52512951]]


**Here we decided to a non-parametric hypothesis to try to understand if the distributions for the adult and offspring are actually the same which is our null hypothesis that the distributions are the same. To do so what we do is create a super data set from both of these distributions and then mix up the values. Then we pull properly sized distributions from this super data set. Then the value we decided could be informative as a test statistic was the difference in the mean values of the two data sets showing how the average beak depth had changed across the generation. For each of the newly created mixed distribution pairs we will take difference of the means. And we want to measure the amount of differences that are at least as extreme as what we observed between the two distributions in the field. The fraction of these values will be our p value.**

In [40]:
# Non-parametric Hypothesis test 

def draw_perm_sample(x, y):
    """Generate a permutation sample."""

    concat_data = np.concatenate((x, y))
    np.random.shuffle(concat_data)

    return concat_data[:len(x)], concat_data[len(x):]


def draw_perm_reps(x, y, stat_fun, size=1):
    """Generate array of permuation replicates."""

    return np.array([stat_fun(*draw_perm_sample(x, y)) for _ in range(size)])

def draw_perm_reps_diff_mean(x, y, size=1):
    """Generate array of permuation replicates."""
    
    out = np.empty(size)
    for i in range(size):
        x_perm, y_perm = draw_perm_sample(x, y)
        out[i] = np.mean(x_perm) - np.mean(y_perm)

    return out

def nhst(df):
    diff_mean = np.mean(df["mid_parent"].values) - np.mean(df["mid_offspring"])
    perm_reps = draw_perm_reps_diff_mean(df["mid_parent"].values,
                                         df["mid_offspring"], size=100000)

    return np.sum(perm_reps >= diff_mean) / len(perm_reps)

print("The p-value of Non-parametric Hypothesis test for the means of the parent and offspring generations being equal are:")
print(nhst(fortis_df), " for fortis and")
print(nhst(scandens_df), " for scandens.")

The p-value of Non-parametric Hypothesis test for the means of the parent and offspring generations being equal are:
9e-05  for fortis and
0.00139  for scandens.


**Given that the p values obtained via this method are negiglibly small. We should reject the null hypothesis that the distributions are the same.** 

Attributions: The coding was done in part by Sophia and in part by Juan as were comments.

In [29]:
%load_ext watermark
%watermark -v -p pandas,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.8.16
IPython version      : 7.9.0

pandas    : 1.3.5
bokeh     : 2.3.3
jupyterlab: not installed

